In [0]:
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
from torch.autograd import Variable
import torchvision.utils as utils
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torch.utils.data as data
import torch.cuda as cuda

In [0]:
#Setting the hyper parameters
batch_size=64
image_size=64

In [11]:
#Image rescaling
transform=transforms.Compose([transforms.Scale(size=image_size),transforms.ToTensor(),transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

/usr/local/lib/python3.6/dist-packages/torchvision/transforms/transforms.py:188: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")


In [12]:
#loading the cifar10 dataset
dataset=dset.CIFAR10(root='./data',download=True,transform=transform)

In [0]:
#implementing dataloader
dataloader=data.DataLoader(dataset,batch_size=batch_size,shuffle=True,num_workers=2)

In [0]:
#weight intialization
def weights_init(m):
    classname=m.__class__.__name__
    if classname.find('conv')!=-1:
        m.weight.data.normal_(0.0, 0.02).cuda()
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02).cuda()
        m.bias.data.fill_(0).cuda()

In [0]:
#Architecture of Generator
class G(nn.Module):
    def __init__(self):
        super(G,self).__init__()
        self.main=nn.Sequential(
            nn.ConvTranspose2d(100,512,4,1,0,bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(True),
            nn.ConvTranspose2d(512,256,4,2,1,bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.ConvTranspose2d(256,128,4,2,1,bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.ConvTranspose2d(128,64,4,2,1,bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.ConvTranspose2d(64,3,4,2,1,bias=False),
            nn.Tanh()                       
        )
        
    def forward(self,input):
        output=self.main(input)
        return output   
        

In [36]:
#creating the generator
netG=G()
netG.apply(weights_init)
netG.cuda()

G(
  (main): Sequential(
    (0): ConvTranspose2d (100, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True)
    (2): ReLU(inplace)
    (3): ConvTranspose2d (512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
    (5): ReLU(inplace)
    (6): ConvTranspose2d (256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
    (8): ReLU(inplace)
    (9): ConvTranspose2d (128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    (11): ReLU(inplace)
    (12): ConvTranspose2d (64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (13): Tanh()
  )
)

In [0]:
#Architecture of Discriminator
class D(nn.Module):
    def __init__(self):
        super(D,self).__init__()
        self.main=nn.Sequential(
            nn.Conv2d(3,64,4,2,1,bias=False),
            nn.LeakyReLU(0.2, inplace = True),
            nn.Conv2d(64,128,4,2,1,bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace = True),
            nn.Conv2d(128,256,4,2,1,bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace = True),
            nn.Conv2d(256,512,4,2,1,bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace = True),
            nn.Conv2d(512,1,4,1,0,bias=False),
            nn.Sigmoid()        
        )

    def forward(self,input):
        output=self.main(input)
        return output.view(-1)


In [37]:
#creating the Descriminator
netD=D()
netD.apply(weights_init)
netD.cuda()

D(
  (main): Sequential(
    (0): Conv2d (3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU(0.2, inplace)
    (2): Conv2d (64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
    (4): LeakyReLU(0.2, inplace)
    (5): Conv2d (128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
    (7): LeakyReLU(0.2, inplace)
    (8): Conv2d (256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (9): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True)
    (10): LeakyReLU(0.2, inplace)
    (11): Conv2d (512, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (12): Sigmoid()
  )
)

In [0]:
#optimizer and loss function
loss=nn.BCELoss()
optimizerG=optim.Adam(netG.parameters(),lr = 0.0002, betas = (0.5, 0.999))
optimizerD=optim.Adam(netD.parameters(),lr = 0.0002, betas = (0.5, 0.999))

In [42]:
#Training the DCGANS

for epoch in range(25):
    for i,data in enumerate(dataloader,0):
        
        #Step 1 training the decriminator
        netD.zero_grad()
        
        #training the decrimator with real image dataset
        real,_=data
        input=Variable(real).cuda()
        target=Variable(torch.ones(input.size()[0])).cuda()
        output=netD(input).cuda()
        errD_real=loss(output,target).cuda()
        
        #training the descrimator with fake image
        noise=Variable(torch.randn(input.size()[0],100,1,1)).cuda()
        fake=netG(noise).cuda()
        target=Variable(torch.zeros(input.size()[0])).cuda()
        output=netD(fake.detach()).cuda()
        errG_fake=loss(output,target).cuda()
        
        #Backpropagation for descriminator
        errD=errD_real+errG_fake
        errD.backward()
        optimizerD.step()
        
        
        #step 2 training the Generator
        netG.zero_grad()
        
        #Traing and applying Backprogation for generator
        target=Variable(torch.ones(input.size()[0])).cuda()
        output=netD(fake).cuda()
        errG=loss(output,target).cuda()
        errG.backward()
        optimizerG.step()
        
        
        print('[%d/%d][%d/%d] loss_D %.4f loss_G %.4f'%(epoch,25,i,len(dataloader),errD[0],errG[0]))
        
        if i %100==0:
          utils.save_image(real,'%s/real_samples.png' % "results", normalize = True)
          fake=netG(noise).cuda()
          utils.save_image(fake.data,'%s/fake_samples_epoch_%03d.png' % ("results", epoch), normalize = True)


[0/25][0/782] loss_D 0.0953 loss_G 3.9243
[0/25][1/782] loss_D 3.4867 loss_G 2.7326
[0/25][2/782] loss_D 2.8715 loss_G 4.5597
[0/25][3/782] loss_D 1.7225 loss_G 6.0353
[0/25][4/782] loss_D 1.4911 loss_G 5.2897
[0/25][5/782] loss_D 2.0070 loss_G 6.3518
[0/25][6/782] loss_D 1.2988 loss_G 6.3060
[0/25][7/782] loss_D 1.2155 loss_G 7.3483
[0/25][8/782] loss_D 0.8722 loss_G 7.1084
[0/25][9/782] loss_D 1.0775 loss_G 8.2928
[0/25][10/782] loss_D 0.7354 loss_G 6.6249
[0/25][11/782] loss_D 1.8608 loss_G 9.3699
[0/25][12/782] loss_D 0.4685 loss_G 7.9848
[0/25][13/782] loss_D 0.8979 loss_G 8.1177
[0/25][14/782] loss_D 0.6118 loss_G 7.9440
[0/25][15/782] loss_D 1.0891 loss_G 9.8403
[0/25][16/782] loss_D 0.5227 loss_G 7.7458
[0/25][17/782] loss_D 1.5467 loss_G 10.1166
[0/25][18/782] loss_D 0.5294 loss_G 9.2407
[0/25][19/782] loss_D 0.4143 loss_G 5.0469
[0/25][20/782] loss_D 3.0958 loss_G 9.7374
[0/25][21/782] loss_D 0.4318 loss_G 10.2435
[0/25][22/782] loss_D 0.3206 loss_G 8.6326
[0/25][23/782] loss

[0/25][189/782] loss_D 0.7991 loss_G 3.0034
[0/25][190/782] loss_D 0.7132 loss_G 2.1174
[0/25][191/782] loss_D 0.8750 loss_G 3.1755
[0/25][192/782] loss_D 0.7215 loss_G 2.0081
[0/25][193/782] loss_D 0.8525 loss_G 3.9299
[0/25][194/782] loss_D 0.8905 loss_G 1.6799
[0/25][195/782] loss_D 1.0416 loss_G 3.4152
[0/25][196/782] loss_D 0.7957 loss_G 3.3952
[0/25][197/782] loss_D 0.7433 loss_G 4.5040
[0/25][198/782] loss_D 0.8948 loss_G 2.2464
[0/25][199/782] loss_D 0.6554 loss_G 3.5897
[0/25][200/782] loss_D 0.4130 loss_G 3.2260
[0/25][201/782] loss_D 0.5897 loss_G 3.9225
[0/25][202/782] loss_D 0.6029 loss_G 3.5887
[0/25][203/782] loss_D 0.6936 loss_G 3.7692
[0/25][204/782] loss_D 0.5621 loss_G 3.5850
[0/25][205/782] loss_D 0.6483 loss_G 3.6420
[0/25][206/782] loss_D 0.4082 loss_G 3.6810
[0/25][207/782] loss_D 0.4958 loss_G 3.2813
[0/25][208/782] loss_D 0.4299 loss_G 4.0339
[0/25][209/782] loss_D 0.5167 loss_G 4.0490
[0/25][210/782] loss_D 0.8391 loss_G 4.1534
[0/25][211/782] loss_D 0.9314 lo

[0/25][376/782] loss_D 0.8788 loss_G 2.7324
[0/25][377/782] loss_D 0.9577 loss_G 2.4582
[0/25][378/782] loss_D 1.1483 loss_G 1.4273
[0/25][379/782] loss_D 1.3785 loss_G 3.5306
[0/25][380/782] loss_D 1.4462 loss_G 1.9240
[0/25][381/782] loss_D 0.6820 loss_G 1.6265
[0/25][382/782] loss_D 0.7526 loss_G 2.4442
[0/25][383/782] loss_D 0.4576 loss_G 2.7329
[0/25][384/782] loss_D 0.5015 loss_G 2.3061
[0/25][385/782] loss_D 0.6015 loss_G 2.6824
[0/25][386/782] loss_D 0.6377 loss_G 2.4053
[0/25][387/782] loss_D 1.0888 loss_G 1.2718
[0/25][388/782] loss_D 1.6016 loss_G 3.8597
[0/25][389/782] loss_D 1.5130 loss_G 1.6741
[0/25][390/782] loss_D 1.0939 loss_G 1.4444
[0/25][391/782] loss_D 1.1771 loss_G 2.6553
[0/25][392/782] loss_D 0.8968 loss_G 2.0643
[0/25][393/782] loss_D 1.0521 loss_G 2.8201
[0/25][394/782] loss_D 1.2860 loss_G 1.2870
[0/25][395/782] loss_D 0.9045 loss_G 3.2022
[0/25][396/782] loss_D 0.6663 loss_G 2.2540
[0/25][397/782] loss_D 0.8609 loss_G 3.0174
[0/25][398/782] loss_D 0.6568 lo

[0/25][654/782] loss_D 1.2901 loss_G 3.0884
[0/25][655/782] loss_D 0.9430 loss_G 1.9777
[0/25][656/782] loss_D 0.6540 loss_G 2.2739
[0/25][657/782] loss_D 0.7126 loss_G 2.3133
[0/25][658/782] loss_D 0.6623 loss_G 2.1772
[0/25][659/782] loss_D 0.6696 loss_G 2.5864
[0/25][660/782] loss_D 0.7144 loss_G 2.0848
[0/25][661/782] loss_D 0.6674 loss_G 1.8729
[0/25][662/782] loss_D 0.9064 loss_G 3.2973
[0/25][663/782] loss_D 0.7280 loss_G 2.0794
[0/25][664/782] loss_D 0.7950 loss_G 2.5110
[0/25][665/782] loss_D 0.7499 loss_G 2.9887
[0/25][666/782] loss_D 0.6224 loss_G 2.6866
[0/25][667/782] loss_D 0.7072 loss_G 2.1443
[0/25][668/782] loss_D 0.6642 loss_G 3.4523
[0/25][669/782] loss_D 0.6480 loss_G 1.9697
[0/25][670/782] loss_D 0.5810 loss_G 3.7149
[0/25][671/782] loss_D 0.5070 loss_G 2.8214
[0/25][672/782] loss_D 0.5888 loss_G 2.7434
[0/25][673/782] loss_D 0.8447 loss_G 3.7376
[0/25][674/782] loss_D 1.2412 loss_G 1.5868
[0/25][675/782] loss_D 1.1387 loss_G 3.2168
[0/25][676/782] loss_D 0.7086 lo

[1/25][149/782] loss_D 1.1614 loss_G 4.7007
[1/25][150/782] loss_D 0.9412 loss_G 2.0983
[1/25][151/782] loss_D 0.6396 loss_G 2.0389
[1/25][152/782] loss_D 0.7716 loss_G 3.7197
[1/25][153/782] loss_D 1.1018 loss_G 1.2282
[1/25][154/782] loss_D 0.9772 loss_G 3.6104
[1/25][155/782] loss_D 0.7489 loss_G 2.6876
[1/25][156/782] loss_D 0.5670 loss_G 2.2582
[1/25][157/782] loss_D 0.9949 loss_G 2.9475
[1/25][158/782] loss_D 0.7576 loss_G 2.1893
[1/25][159/782] loss_D 0.8030 loss_G 3.0083
[1/25][160/782] loss_D 0.5408 loss_G 2.6360
[1/25][161/782] loss_D 0.4199 loss_G 3.0148
[1/25][162/782] loss_D 0.4840 loss_G 2.8727
[1/25][163/782] loss_D 0.5245 loss_G 2.7529
[1/25][164/782] loss_D 0.7273 loss_G 2.5259
[1/25][165/782] loss_D 0.6861 loss_G 3.1362
[1/25][166/782] loss_D 0.6638 loss_G 2.3885
[1/25][167/782] loss_D 0.6874 loss_G 2.4493
[1/25][168/782] loss_D 0.5856 loss_G 2.9076
[1/25][169/782] loss_D 0.7267 loss_G 2.1580
[1/25][170/782] loss_D 0.6144 loss_G 2.6608
[1/25][171/782] loss_D 0.6730 lo

...

[23/25][533/782] loss_D 0.3792 loss_G 3.0164
[23/25][534/782] loss_D 0.5236 loss_G 3.7825
[23/25][535/782] loss_D 0.3750 loss_G 2.5125
[23/25][536/782] loss_D 0.2091 loss_G 3.5039
[23/25][537/782] loss_D 0.2660 loss_G 3.6981
[23/25][538/782] loss_D 0.2548 loss_G 3.2859
[23/25][539/782] loss_D 0.3566 loss_G 3.4176
[23/25][540/782] loss_D 0.2427 loss_G 2.7593
[23/25][541/782] loss_D 0.5161 loss_G 6.2007
[23/25][542/782] loss_D 0.5867 loss_G 2.5578
[23/25][543/782] loss_D 0.1586 loss_G 2.7086
[23/25][544/782] loss_D 0.5631 loss_G 6.3252
[23/25][545/782] loss_D 0.5245 loss_G 2.8090
[23/25][546/782] loss_D 0.2327 loss_G 3.0696
[23/25][547/782] loss_D 0.2570 loss_G 5.2058
[23/25][548/782] loss_D 0.3650 loss_G 2.1034
[23/25][549/782] loss_D 0.2264 loss_G 4.4441
[23/25][550/782] loss_D 0.1259 loss_G 5.0084
[23/25][551/782] loss_D 0.3954 loss_G 2.2174
[23/25][552/782] loss_D 0.3904 loss_G 5.0213
[23/25][553/782] loss_D 0.1845 loss_G 3.7044
[23/25][554/782] loss_D 0.1438 loss_G 3.6643
[23/25][55

[23/25][717/782] loss_D 1.1925 loss_G 1.7847
[23/25][718/782] loss_D 0.4256 loss_G 2.8292
[23/25][719/782] loss_D 0.2970 loss_G 3.6951
[23/25][720/782] loss_D 0.4250 loss_G 3.3936
[23/25][721/782] loss_D 0.2967 loss_G 2.6684
[23/25][722/782] loss_D 0.3118 loss_G 4.5601
[23/25][723/782] loss_D 0.3565 loss_G 2.8178
[23/25][724/782] loss_D 0.3261 loss_G 3.7234
[23/25][725/782] loss_D 0.2127 loss_G 4.1354
[23/25][726/782] loss_D 0.2325 loss_G 4.8005
[23/25][727/782] loss_D 0.6671 loss_G 2.1392
[23/25][728/782] loss_D 0.3569 loss_G 4.1349
[23/25][729/782] loss_D 0.8591 loss_G 0.1535
[23/25][730/782] loss_D 3.7304 loss_G 8.9497
[23/25][731/782] loss_D 3.5585 loss_G 1.8693
[23/25][732/782] loss_D 0.4342 loss_G 1.3920
[23/25][733/782] loss_D 0.8322 loss_G 4.5830
[23/25][734/782] loss_D 0.8962 loss_G 1.4976
[23/25][735/782] loss_D 0.7465 loss_G 1.7781
[23/25][736/782] loss_D 0.5984 loss_G 3.6869
[23/25][737/782] loss_D 0.8431 loss_G 1.9312
[23/25][738/782] loss_D 0.4356 loss_G 2.9360
[23/25][73

[24/25][209/782] loss_D 0.5278 loss_G 2.2957
[24/25][210/782] loss_D 0.4520 loss_G 2.6672
[24/25][211/782] loss_D 0.3839 loss_G 2.8763
[24/25][212/782] loss_D 0.4772 loss_G 2.4072
[24/25][213/782] loss_D 0.8443 loss_G 1.9317
[24/25][214/782] loss_D 0.6764 loss_G 3.5500
[24/25][215/782] loss_D 0.7341 loss_G 1.7871
[24/25][216/782] loss_D 0.5563 loss_G 3.6519
[24/25][217/782] loss_D 0.5070 loss_G 3.0066
[24/25][218/782] loss_D 0.6500 loss_G 1.3785
[24/25][219/782] loss_D 0.9278 loss_G 3.7728
[24/25][220/782] loss_D 0.4674 loss_G 2.9750
[24/25][221/782] loss_D 0.4373 loss_G 3.2109
[24/25][222/782] loss_D 0.5626 loss_G 1.9143
[24/25][223/782] loss_D 0.4923 loss_G 3.1458
[24/25][224/782] loss_D 0.4539 loss_G 2.9933
[24/25][225/782] loss_D 0.3664 loss_G 2.7116
[24/25][226/782] loss_D 0.3049 loss_G 2.9999
[24/25][227/782] loss_D 0.3497 loss_G 2.3281
[24/25][228/782] loss_D 0.4731 loss_G 3.5192
[24/25][229/782] loss_D 0.5484 loss_G 2.6690
[24/25][230/782] loss_D 0.5035 loss_G 2.6055
[24/25][23

[24/25][393/782] loss_D 0.2135 loss_G 6.5978
[24/25][394/782] loss_D 0.1235 loss_G 5.4996
[24/25][395/782] loss_D 0.0195 loss_G 5.4426
[24/25][396/782] loss_D 0.0058 loss_G 6.0006
[24/25][397/782] loss_D 0.0614 loss_G 4.6344
[24/25][398/782] loss_D 0.0244 loss_G 5.0087
[24/25][399/782] loss_D 0.0200 loss_G 5.9968
[24/25][400/782] loss_D 0.0257 loss_G 5.0379
[24/25][401/782] loss_D 0.0154 loss_G 5.8370
[24/25][402/782] loss_D 0.0284 loss_G 4.6522
[24/25][403/782] loss_D 0.0218 loss_G 5.7050
[24/25][404/782] loss_D 0.0118 loss_G 6.9575
[24/25][405/782] loss_D 0.0102 loss_G 5.7034
[24/25][406/782] loss_D 0.0334 loss_G 4.7599
[24/25][407/782] loss_D 0.0416 loss_G 4.3660
[24/25][408/782] loss_D 0.0201 loss_G 5.2299
[24/25][409/782] loss_D 0.0513 loss_G 4.7439
[24/25][410/782] loss_D 0.0399 loss_G 5.2573
[24/25][411/782] loss_D 0.0950 loss_G 6.3817
[24/25][412/782] loss_D 0.0388 loss_G 4.3718
[24/25][413/782] loss_D 0.0145 loss_G 6.1479
[24/25][414/782] loss_D 0.0073 loss_G 5.9927
[24/25][41

[24/25][576/782] loss_D 0.4349 loss_G 2.6701
[24/25][577/782] loss_D 0.6648 loss_G 2.6301
[24/25][578/782] loss_D 0.5747 loss_G 2.5490
[24/25][579/782] loss_D 0.4505 loss_G 3.0773
[24/25][580/782] loss_D 0.4167 loss_G 3.1988
[24/25][581/782] loss_D 0.5254 loss_G 2.4687
[24/25][582/782] loss_D 0.5514 loss_G 3.6030
[24/25][583/782] loss_D 0.6281 loss_G 2.3900
[24/25][584/782] loss_D 0.6598 loss_G 3.2506
[24/25][585/782] loss_D 0.6693 loss_G 2.1258
[24/25][586/782] loss_D 0.6345 loss_G 2.7145
[24/25][587/782] loss_D 0.5637 loss_G 3.0219
[24/25][588/782] loss_D 0.5392 loss_G 3.6036
[24/25][589/782] loss_D 0.6161 loss_G 1.9603
[24/25][590/782] loss_D 0.5505 loss_G 2.9512
[24/25][591/782] loss_D 0.5212 loss_G 3.4403
[24/25][592/782] loss_D 0.7497 loss_G 1.3224
[24/25][593/782] loss_D 0.9667 loss_G 5.2587
[24/25][594/782] loss_D 0.4729 loss_G 3.8070
[24/25][595/782] loss_D 0.6312 loss_G 1.6803
[24/25][596/782] loss_D 0.8477 loss_G 3.9767
[24/25][597/782] loss_D 0.6907 loss_G 3.0711
[24/25][59

In [46]:
torch.save(netG,'model_generator.pt')
torch.save(netD,'model_descriminator.pt')

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:158: UserWarning: Couldn't retrieve source code for container of type G. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:158: UserWarning: Couldn't retrieve source code for container of type D. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
